<h1><center>Лабораторна робота 6.</center></h1>
<h2><center>Лінійна регресія, Lasso і RF-регресія для задачі визначення якості вина</center></h2>

**Виконав:** Прізвище І.П.

**Варіант:** №__

<a class="anchor" id="lab-6"></a>

## Зміст

- [6.1. Завантаження навчальних даних](#lab-6.1)
- [6.2. Лінійна регресія](#lab-6.2)  
- [6.3. LASSO-регресія](#lab-6.3)
- [6.4. Випадковий ліс](#lab-6.4)
- [6.5. Порівняння моделей](#lab-6.5)

In [1]:
# відключимо попередження Anaconda
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LinearRegression, LassoCV, Lasso
from sklearn.ensemble import RandomForestRegressor

<a class="anchor" id="lab-6.1"></a>

## <span style="color:blue; font-size:1.2em;">6.1. Завантаження навчальних даних</span>

[Повернутися до змісту](#lab-6)

В цій лабораторній робота будемо працювати з набором даних за якістю білого вина ([репозиторій UCI](https://archive.ics.uci.edu/ml/datasets/wine+quality)).

Спершу завантажуємо дані:

In [2]:
winequality_white_url = 'https://raw.githubusercontent.com/radiukpavlo/intelligent-data-analysis/main/02_assignments/ida_lab-06_regression-wine/winequality-white.csv'
data = pd.read_csv(winequality_white_url, sep=';')

In [3]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         4898 non-null   float64
 1   volatile acidity      4898 non-null   float64
 2   citric acid           4898 non-null   float64
 3   residual sugar        4898 non-null   float64
 4   chlorides             4898 non-null   float64
 5   free sulfur dioxide   4898 non-null   float64
 6   total sulfur dioxide  4898 non-null   float64
 7   density               4898 non-null   float64
 8   pH                    4898 non-null   float64
 9   sulphates             4898 non-null   float64
 10  alcohol               4898 non-null   float64
 11  quality               4898 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


Виконаємо відокремлення цільової ознаки. Для цього розділимо навчальну вибірку за відношенням 7:3; тут 30% даних - валідаційна вибірка. Задамо `random_state=17` і виконаємо стандартизацію даних за допомогою `StandardScaler()`.

In [5]:
y = data['quality']
X = data.drop('quality', axis=1)

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=17)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)

<a class="anchor" id="lab-6.2"></a>

## <span style="color:blue; font-size:1.2em;">6.2. Лінійна регресія</span>

[Повернутися до змісту](#lab-6)

Виконайте навчання простої лінійної регресії. Пам'ятейте, що для навчання та тестування моделі ми використовуємо стандартизовані значення незалежних змінних `X_train_scaled` та `X_valid_scaled`, і початкові значення цільової ознаки `y_train` та `y_valid`.

In [6]:
linreg = LinearRegression(normalize=True)
linreg.fit(X_train_scaled,y_train)

LinearRegression(normalize=True)

Після навчання моделі перегляньте її коефіцієнти й відсортуйте ознаки за впливом на якість вина. Майте на увазі, що великі за модулем негативні значення коефіцієнтів теж вказують на сильний вплив на цільову ознаку. Для виконання цього завдання створіть новий DataFrame, як в попередній темі.

In [7]:
linreg_coef = pd.DataFrame(linreg.coef_,data.columns[:-1], columns = ['coeff'])
linreg_coef.sort_values(by = 'coeff', ascending=False)

,coeff
residual sugar,0.538164
pH,0.150036
alcohol,0.129533
fixed acidity,0.097822
sulphates,0.062053
free sulfur dioxide,0.042180
total sulfur dioxide,0.014304
chlorides,0.008127
citric acid,-0.000183
volatile acidity,-0.192260


### <span style="color:red; font-size:1.5em;">Завдання 1</span>


**Для варіантів 1-5:**

Яка ознака в моделі лінійної регресії має найменший вплив на якість вина?

**Для варіантів 6-10:**

Обрахуйте середньоквадратичні помилки лінійної регресії за навчальною та валідаційною вибірками.

**Для варіантів 11-15:**

Обрахуйте середньоквадратичні помилки лінійної регресії за навчальною та валідаційною вибірками. 

**Для варіантів 16-20:**

Яка ознака в моделі лінійної регресії має найбільший вплив на якість вина?

In [11]:
print("Середньоквадратична помилка (train): %.3f" % mean_squared_error(y_train, linreg.predict(X_train_scaled)))
print("Середньоквадратична помилка (valid): %.3f" % mean_squared_error(y_valid, linreg.predict(X_valid_scaled)))

Середньоквадратична помилка (train): 0.558
Середньоквадратична помилка (valid): 0.584


<a class="anchor" id="lab-6.3"></a>

## <span style="color:blue; font-size:1.2em;">6.3. Lasso-регресія</span>

[Повернутися до змісту](#lab-6)

Навчіть Lasso-регресію з невеликим коефіцієнтом $\alpha = 0.01$ (слабка регуляризація). Прийміть `random_state=17` та `cv=5`.

In [12]:
lasso1 = Lasso(alpha=0.01, random_state=17)
lasso1.fit(X_train_scaled, y_train)

Lasso(alpha=0.01, random_state=17)

Обрахуйте вплив всіх незалежних ознак на цільову ознаку LASSO-регресії. Відсортуйте ознаки за впливом на якість вина.

In [16]:
lasso1_coef = pd.DataFrame(lasso1.coef_,data.columns[:-1], columns = ['coeff'])
lasso1_coef.sort_values(by = 'coeff', ascending = False)

,coeff
alcohol,0.322425
residual sugar,0.256363
pH,0.067277
free sulfur dioxide,0.043088
sulphates,0.029722
fixed acidity,-0.000000
citric acid,-0.000000
total sulfur dioxide,-0.000000
chlorides,-0.002747
volatile acidity,-0.188479


Код нижче дасть змогу знайти найкраще значення $\alpha$ поміж заданих.

In [ ]:
alphas = np.logspace(-6, 2, 200)
lasso_cv = LassoCV # Ваш код тут
lasso_cv.fit # Ваш код тут

### <span style="color:red; font-size:1.5em;">Завдання 2</span>


**Для варіантів 1-5:**

Обрахуйте середньоквадратичні помилки побудованої LASSO-регресії за навчальною та валідаційною вибірками. 

**Для варіантів 6-10:**

Яка ознака в моделі LASSO-регресії має найбільший вплив на якість вина? Використайте підхід з DataFrame як в попередньому завданні.

**Для варіантів 11-15:**

Обрахуйте найкраще значення $\alpha$ для LASSO-регресії за 5-кратною крос-валідацією. Використайте LassoCV, `random_state = 17` і команду `alpha_`.

**Для варіантів 16-20:**

Обрахуйте середньоквадратичні помилки побудованої LASSO-регресії за навчальною та валідаційною вибірками. 

<a class="anchor" id="lab-6.4"></a>

## <span style="color:blue; font-size:1.2em;">6.4. Випадковий ліс</span>

[Повернутися до змісту](#lab-6)

Навчіть випадковий ліс з параметрами "з коробки", фіксуючи тільки `random_state=17`.

In [18]:
rf = RandomForestRegressor(random_state=17)
rf.fit(X_train, y_train)


RandomForestRegressor(random_state=17)

### <span style="color:red; font-size:1.5em;">Завдання 3</span>

**Для варіантів 1-5:**

Обрахуйте середньоквадратичні помилки побудованого випадкового лісу за навчальною та валідаційною вибірками за допомогою GridSearchCV(). Протестуйте значення параметра `'n_estimators': [10, 50, 100, 250, 500]`. 

**Для варіантів 6-10:**

Обрахуйте середньоквадратичні помилки побудованого випадкового лісу за навчальною та валідаційною вибірками за допомогою GridSearchCV(). Протестуйте значення параметра `'max_features': [3, 4, 5, 7, 9]`. 

**Для варіантів 11-15:**

Обрахуйте середньоквадратичні помилки побудованого випадкового лісу за навчальною та валідаційною вибірками за допомогою GridSearchCV(). Протестуйте значення параметра `min_samples_leaf': [1, 3, 5, 7, 9]`. 

**Для варіантів 16-20:**

Обрахуйте середньоквадратичні помилки побудованого випадкового лісу за навчальною та валідаційною вибірками за допомогою GridSearchCV(). Протестуйте значення параметра `'max_depth': [5, 7, 9, 11, 13]`.  

Середньоквадратична помилка на крос-валідації (cross_val_score з `scoring='neg_mean_squared_error'`) буде такою:

In [19]:
print("Середньоквадратична помилка (train): %.3f" % mean_squared_error(y_train, rf.predict(X_train)))
print("Середньоквадратична помилка (valid): %.3f" % mean_squared_error(y_valid, rf.predict(X_valid)))

Середньоквадратична помилка (train): 0.053
Середньоквадратична помилка (valid): 0.371


Далі налаштуйте всі важливі параметри випадкового лісу за допомогою GridSearchCV(). Тюнинг одночасно кількох параметрів може зайняти чимало часу на слабких процесорах, тому будьте готові до тривалих обрахунків. 

In [20]:
rf_params = {'n_estimators': [10, 50],
             'criterion': ['squared_error', 'absolute_error', 'poisson'],
             'max_features': [3, 5],
             'min_samples_leaf': [3, 5],
             'max_depth': [5, 7]}

In [21]:
%%time
locally_best_rf = GridSearchCV(rf, rf_params,n_jobs=-1)
locally_best_rf.fit(X_train, y_train)

CPU times: user 2.57 s, sys: 195 ms, total: 2.76 s
Wall time: 2min 23s


GridSearchCV(estimator=RandomForestRegressor(random_state=17), n_jobs=-1,
             param_grid={'criterion': ['squared_error', 'absolute_error',
                                       'poisson'],
                         'max_depth': [5, 7], 'max_features': [3, 5],
                         'min_samples_leaf': [3, 5], 'n_estimators': [10, 50]})

In [22]:
locally_best_rf.best_params_, locally_best_rf.best_score_


({'criterion': 'squared_error',
  'max_depth': 7,
  'max_features': 5,
  'min_samples_leaf': 3,
  'n_estimators': 50},
 0.38190651014361565)

Насамкінець, оцініть важливість всіх ознак налаштованого випадкового лісу.

In [23]:

rf_importance = pd.DataFrame(rf.feature_importances_, data.columns[:-1], columns = ['coeff'])
rf_importance.sort_values(by = 'coeff', ascending = True)

,coeff
density,0.056540
citric acid,0.058904
sulphates,0.059056
fixed acidity,0.061918
chlorides,0.063729
total sulfur dioxide,0.067204
residual sugar,0.070903
pH,0.071950
free sulfur dioxide,0.116562
volatile acidity,0.125960


### <span style="color:red; font-size:1.5em;">Завдання 4</span>


**Для варіантів 1-5:**

Обрахуйте середньоквадратичні помилки налаштованого випадкового лісу за навчальною та валідаційною вибірками та на крос-валідації (cross_val_score з `scoring='neg_mean_squared_error'`). Скористайтеся знайденими значення параметрів за допомогою GridSearchCV з попереднього кроку.

**Для варіантів 6-10:**

Яка ознака виявилася найгіршою (з найменшим значенням) для моделі випадкового лісу? 

**Для варіантів 11-15:**

Яка ознака виявилася найкращою (з найбільшим значенням) для моделі випадкового лісу?

**Для варіантів 16-20:**

Обрахуйте середньоквадратичні помилки налаштованого випадкового лісу за навчальною та валідаційною вибірками та на крос-валідації (cross_val_score з `scoring=neg_mean_squared_log_error'`). Скористайтеся знайденими значення параметрів за допомогою GridSearchCV з попереднього кроку.

In [33]:
rf_importance.sort_values(by = 'coeff', ascending = True).head(1)

,coeff
density,0.05654


## <span style="color:blue; font-size:1.2em;">6.5. Порівняння моделей</span>

[Повернутися до змісту](#lab-6)

### <span style="color:red; font-size:1.5em;">Завдання 5</span>

**Для всіх варіантів:**

Зробіть висновки щодо якості побудованих моделей: лінійна регресія, LASSO-регресія та випадковий ліс. Охорайтеризуйте знайдені оцінки впливу ознак на якість вина.

Висновки:
  Середньоквадратичні помилки моделі лінійної регресії та LASSO-регресії майже однакові: навчальна вибірка (0.558 і 0.564), тестова вибірка (0.584 і 0.574).
З цього можна робити висновки, щодо якості побудованих моделей.
  В свою чергу випадковий ліс має значно менші середньоквадратичні помилки відносно своїх конкурентів на обох вибірках: навчальна вибірка (0.053) та тестова вибірка (0.371).

Щодо ознак, що впливали на якість вина:
*   У лінійної регресії найбільше впливало - residual sugar, а наймешне - density.
*   У LASSO-регресії найбільше впливало - alcohol, а наймешне - density.
*   У випадкового лісу найбільше впливало - alcohol, а наймешне - density.